In [1]:
import tensorflow 

import pandas as pd
import numpy as np
import os
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import VGG19, DenseNet169
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

'''Reference:
    https://www.pluralsight.com/guides/introduction-to-densenet-with-tensorflow
'''

'Reference:\n    https://www.pluralsight.com/guides/introduction-to-densenet-with-tensorflow\n'

In [2]:
print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.9.1


In [3]:
model_d=VGG19(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) 

x=model_d.output

x= GlobalAveragePooling2D()(x)
x= BatchNormalization()(x)
x= Dropout(0.5)(x)
x= Dense(1024,activation='relu')(x) 
x= Dense(512,activation='relu')(x) 
x= BatchNormalization()(x)
x= Dropout(0.5)(x)

preds=Dense(3,activation='softmax')(x) #FC-layer, your number of the classes

In [4]:
model=Model(inputs=model_d.input,outputs=preds)
model.summary()



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [5]:
for layer in model.layers[:-8]:
    layer.trainable=False
    
for layer in model.layers[-8:]:
    layer.trainable=True
    
#To avoid the problem of overfitting, avoid training the entire network. layer.trainable=False will freeze all the layers, keeping only the last eight layers (FC) to detect edges and blobs in the image. Once the model is fitted well, it can be fine-tuned by using layer.trainable=True.

In [6]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [7]:
data=[]
labels=[]
random.seed(42)

imagePaths = list(os.listdir("C:/UNIFESP_Xray/Xray_train_224/"))  #sorting by numbers

In [8]:
for folder in imagePaths:
    path=list(os.listdir("C:/UNIFESP_Xray/Xray_train_224/"+folder))   #folders are class names
# Get the image array and label of every image

    for i in path:
        image = cv2.imread("C:/UNIFESP_Xray/Xray_train_224/"+str(folder)+'/'+i)
        image = cv2.resize(image, (128,128))
        image = img_to_array(image)
        data.append(image)

    # Change the labels to 3 classes
        l = label = folder
        if folder == "3":
            l = 0
        elif folder == "14" :
            l = 1
        else:
            l = 2
        
        labels.append(l)

print("Count for label 0 : ", labels.count(0))
print("Count for label 1 : ", labels.count(1))
print("Count for label 2 : ", labels.count(2))

Count for label 0 :  724
Count for label 1 :  120
Count for label 2 :  762


In [9]:
# Use the reciprocal of every single class as the weights 

print(pd.DataFrame(labels).value_counts(sort=False))
weights=np.reciprocal(np.array(pd.DataFrame(labels).value_counts(sort=False))/1000) #reciprocal(倒數)needs input less than 1
print("weight list:",weights)


#try sample weight to balance the training

0    724
1    120
2    762
dtype: int64
weight list: [1.38121547 8.33333333 1.31233596]


In [10]:
data = np.array(data, dtype="float32") / 255.0   #normaliztion
labels = np.array(labels)
mlb = LabelBinarizer()
onehot_labels = mlb.fit_transform(labels)     #Binarize labels in a one-vs-all fashion. one-hot encoding
print(onehot_labels)

[[0 0 1]
 [0 0 1]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [11]:
(xtrain,xtest,ytrain,ytest)=train_test_split(data,onehot_labels,test_size=0.2,random_state=42)
sample_weight = []
for i in range(len(ytrain)):
    for j in range(len(weights)):
        if labels[i] == j:
            sample_weight.append(weights[j])

print(xtrain.shape, xtest.shape)

(1284, 128, 128, 3) (322, 128, 128, 3)


In [ ]:
# Reduce learning rate when a metric has stopped improving.
# Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)

# Callback to save the Keras model or model weights at some frequency.
checkpoint = ModelCheckpoint('VGG19_3_weights.h5', verbose=1, save_best_only=True)

# Generate batches of tensor image data with real-time data augmentation.
datagen = ImageDataGenerator(zoom_range = 0.5, horizontal_flip=True, shear_range=0.5)


datagen.fit(xtrain)
# Fits-the-model
history = model.fit_generator(datagen.flow(xtrain, ytrain, batch_size=128, shuffle=True, sample_weight=sample_weight),
               steps_per_epoch=xtrain.shape[0] //128,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(xtrain, ytrain))

Epoch 1/50


In [ ]:
# Generates output predictions for the input samples. Computation is done in batches.
ypred = model.predict(xtest)

total = 0
accurate = 0
accurateindex = []
wrongindex = []

for i in range(len(ypred)):
    if np.argmax(ypred[i]) == np.argmax(ytest[i]):
        accurate += 1
        accurateindex.append(i)
    else:
        wrongindex.append(i)
        
    total += 1
    
print('Total-test-data;', total, '\taccurately-predicted-data:', accurate, '\t wrongly-predicted-data: ', total - accurate)
print('Accuracy:', round(accurate/total*100, 3), '%')

In [ ]:
# Show some random accurately predicted samples
label=['Chest','Others','20 parts']
imidx = random.sample(accurateindex, k=9) #random.sample(sequence, k), k: An Integer value, it specify the length of a sample.


nrows = 3
ncols = 3
fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True,figsize=(15, 12))

n = 0
for row in range(nrows):
    for col in range(ncols):
            ax[row,col].imshow(xtest[imidx[n]])
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(label[np.argmax(ypred[imidx[n]])], label[np.argmax(ytest[imidx[n]])]))
            n += 1

plt.show()

In [ ]:
Ypred = model.predict(xtest)

Ypred = np.argmax(Ypred, axis=1)
Ytrue = np.argmax(ytest, axis=1)

cm = confusion_matrix(Ytrue, Ypred)
plt.figure(figsize=(12, 12))
ax = sns.heatmap(cm, cmap="rocket_r", fmt=".01f",annot_kws={'size':10}, annot=True, square=True, xticklabels=label, yticklabels=label)
ax.set_ylabel('Actual', fontsize=20)
ax.set_xlabel('Predicted', fontsize=20)

In [ ]:
loaded_model = tensorflow.keras.models.load_model('VGG19_3_weights.h5')

In [ ]:
test_pd = pd.read_csv("C:/UNIFESP_Xray/Xray_small/sample_submission.csv")

test_path = "C:/UNIFESP_Xray/Xray_small_jpg/test/"

submission_data = []
submission_labels = []
print(type(submission_data))

for test_id in test_pd['SOPInstanceUID']:
    print(test_id)
    test_img = cv2.imread(test_path+test_id+"-c.jpg")
    test_img = cv2.resize(test_img, (128,128))
    test_img = img_to_array(test_img)
    submission_data.append(test_img)

submission_data = np.array(submission_data, dtype="float32") / 255.0   #normaliztion
    
submission_preds = loaded_model.predict(submission_data)
print("raw preds:",submission_preds)

submission_labels = [label[index].replace("_"," ") for index in np.argmax(submission_preds, axis=1)]

print("argmax:",submission_labels)

test_pd['Target'] = submission_labels
test_pd.to_csv("C:/Users/janic/Desktop/DSFederal/VGG19_3_weights.csv", index=False)